<a href="https://colab.research.google.com/github/shub-codes/HateSpeechDetection/blob/main/Hate_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install kaggle and opendataset libraries to import data from kaggle. then import opendatasets and download using the data link

In [18]:
!pip install kaggle
!pip install opendatasets

In [19]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/thedevastator/hate-speech-and-offensive-language-detection")

Skipping, found downloaded files in "./hate-speech-and-offensive-language-detection" (use force=True to force download)


<h1> 1) importing libraries
<ol>
<li>   pandas - analyse, clean,explore and manipulate the dataset .<br>
<li>  re or regex - A RegEx, or Regular Expression, is a sequence of characters that forms a search pattern. RegEx can be used to check if a string contains the specified search pattern.<br>
<li> nltk- natural language transformer kit.<br>


In [20]:
import pandas as pd #to read and write csv files
import numpy as np
import nltk # natural language toolkit
import re #regular expressions
from nltk.corpus import stopwords # to remove words like a, the,is etc. such words contribute almost nothing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras import backend as K


<h1> 2) Importing and reading data

In [21]:
df=pd.read_csv("/content/hate-speech-and-offensive-language-detection/train.csv")
print(df.head(5))
print(f"df has {df.shape[0]}  rows and {df.shape[1]} cols")

   count  hate_speech_count  offensive_language_count  neither_count  class  \
0      3                  0                         0              3      2   
1      3                  0                         3              0      1   
2      3                  0                         3              0      1   
3      3                  0                         2              1      1   
4      6                  0                         6              0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  
df has 24783  rows and 6 cols


<h1> 3) CLEANING THE DATA USING REGULAR EXPRESSIONS </h1>

In [24]:
stopWords= set(stopwords.words('english'))
stopWords.add('RT') # to add the RT which means retweet
#remove html entity
def remove_entity(raw_text):
  text_regex= r"&(#\d+|#x[a-fA-F0-9]+|[a-zA-Z]+);"
  updated_text=re.sub(text_regex,"",raw_text)
  return updated_text
# removes user tags
def remove_user_tags(raw_text):
  user_regex=r"@\w+"
  updated_text=re.sub(user_regex,"",raw_text)
  return updated_text
# removes urls
def remove_urls(raw_text):
  url_regex=r"https?://\S+|www\.\S+"
  updated_text=re.sub(url_regex,"",raw_text,flags=re.IGNORECASE)
  return updated_text
# to remove unnecessary markings
def remove_noise(raw_text):
  noise_regex=r"['\"!`]|\.{2,}"
  updated_text=re.sub(noise_regex,"",raw_text)
  return updated_text
# to tokenise the string and remove stopwords
def remove_stopwords(raw_text):
  tokenize= nltk.word_tokenize(raw_text)
  updated_text=[word for word in tokenize if word not in stopWords]
  return " ".join(updated_text)
# calling all the above functions
def preprocess(raw_text):
  updated=[]
  updated=[remove_entity(text) for text in raw_text]
  updated=[remove_user_tags(text) for text in updated]
  updated=[remove_urls(text) for text in updated]
  updated=[remove_noise(text) for text in updated]
  updated=[remove_stopwords(text) for text in updated]
  return updated
#clean_tweet=preprocess(tweet)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


<h2> Explaination of regex given above</h2>
<ul>
<li> <h3> HTML func regex = r"&(#\d+|#x[a-fA-F0-9]+|[a-zA-Z]+)</h3>
  <ul> <li>r stands to tell that the text is raw
      <li>  #\d tells for numeric references eg &#12345
      <li> #x[a-fA-F0-9] tells for hexadecimal numeric references
      <li> [a-zA-Z] mathces named entities &amp, &lt
  </ul>
<li><h3> user tags func = r"@\w+"<h3>
  <ul> <li> \w stands for  all alphanumeric letters and _
        <li> example "Hey @john_doe, check this out! Also, @alice123 and @user.test are here." changes to "Hey user, check this out! Also, user and user are here."
  </ul>
<li><h3> url func = r"https?://\S+|www\.\S+"<h3>
  <ul> <li> https? means the s in last is optional. hence it mathces to both http and https.
      <li> \S+ matches any additional string
      <li> www\.\S+ matches for all url in for www.dsfdjskn.com
      <li> flag=re.IGNORECASE means it matches irrespective of case of letters
  </ul>
<li> <h3> remove noise func</h3>
  <ul> <li> removes ", ', !' or two and more dots
  </ul>
